In [2]:
from random import uniform as randFloat
from tqdm import tqdm_notebook as tqdm
from statistics import mean
import ipyvolume.pylab as ipv
import numpy as np
import time
import math

In [48]:
class Bird():
    def __init__(self, id, flock, position, velocity = [0, 0, 0]):
        self.position = position
        self.velocity = velocity
        self.nearestneighbors = []
        self.flock = flock
        self.id = id
        
    def nearme(self):
        """
        determines 7 nearest neighbors
        """
        #close = math.sqrt(math.pow(self.flock.xmax - self.flock.xmin, 2), math.pow(self.flock.ymax - self.flock.ymin, 2), math.pow(self.flock.zmax - self.flock.zmin, 2))
        close = 999999
        closest = [close, close, close, close, close, close, close]
        closestbirds = [0, 0, 0, 0, 0, 0, 0]
        for i in range(7):
            for bird in range(self.flock.size):
                if (bird != self.id):
                    distance = math.sqrt(math.pow(self.position[0] - self.flock.birds[bird].position[0], 2) + math.pow(self.position[1] - self.flock.birds[bird].position[1], 2) + math.pow(self.position[2] - self.flock.birds[bird].position[2], 2))
                    if (bird not in closestbirds):
                        if (distance < closest[i]):
                            closestbirds[i] = bird
                            closest[i] = distance
        self.nearestneighbors.append(closestbirds)
        return closestbirds
    
    def qsnearme(self):
        distFlockBirds = []
        numFlockBirds = []
        for bird in range(self.flock.size):
            distFlockBirds.append(math.sqrt(math.pow(self.position[0] - self.flock.birds[bird].position[0], 2) + math.pow(self.position[1] - self.flock.birds[bird].position[1], 2) + math.pow(self.position[2] - self.flock.birds[bird].position[2], 2)))
        for i in range(self.flock.size):
            numFlockBirds.append(i) 
        l = len(distFlockBirds)
        self.quickSort(distFlockBirds, numFlockBirds, 0, l-1)
        numFlockBirds.remove(self.id)
        self.nearestneighbors = numFlockBirds[:7]
        
    def partition(self, arr, numarr, low, high): 
        i = (low-1)
        pivot = arr[high]
        for j in range(low , high): 
            if   arr[j] <= pivot: 
                i = i+1 
                arr[i],arr[j] = arr[j],arr[i]
                numarr[i], numarr[j] = numarr[j], numarr[i]
        arr[i+1],arr[high] = arr[high],arr[i+1]
        numarr[i+1],numarr[high] = numarr[high],numarr[i+1]
        return (i+1) 

    def quickSort(self, arr, numarr, low, high): 
        if low < high: 
            p = self.partition(arr, numarr, low, high) 
            self.quickSort(arr, numarr, low, p - 1) 
            self.quickSort(arr, numarr, p + 1, high) 
        
    def align(self, variability = 1):
        """
        Aligns itself by determining velocity based on 7 nearest neighbors
        """
        output = [0, 0, 0]
        self.qsnearme()
        for j in range(3):
            output[j] = sum(self.flock.birds[self.nearestneighbors[i]].velocity[j] for i in range(7))/len(self.nearestneighbors) + randFloat(-variability, variability)
        return output

    def separate(self):
        xforces, yforces, zforces = [], [], []
        for bird in self.nearestneighbors:
            distancex, distancey, distancez = self.position[0] - self.flock.birds[bird].position[0], self.position[1] - self.flock.birds[bird].position[1], self.position[2] - self.flock.birds[bird].position[2]
            xforce, yforce, zforce = -1/distancex, -1/distancey, -1/distancez
            xforces.append(xforce)
            yforces.append(yforce)
            zforces.append(zforce)
        output = [mean(xforces), mean(yforces), mean(zforces)]
        maximum = 0
        for i in output:
            if(abs(i) > maximum):
                maximum = abs(i)
        output[0] = output[0] / maximum
        output[1] = output[1] / maximum
        output[2] = output[2] / maximum
        return output
    
    def wallforce(self):
        xforces, yforces, zforces = [], [], []
        for bird in self.nearestneighbors:
            distancexmin, distanceymin, distancezmin = self.position[0] - self.flock.xmin, self.position[1] - self.flock.ymin, self.position[2] - self.flock.zmin
            distancexmax, distanceymax, distancezmax = self.flock.xmax - self.position[0], self.flock.ymax - self.position[1], self.flock.zmax - self.position[2]
            xfactor, yfactor, zfactor = 1, 1, 1
            if(distancexmax <= distancexmin):
                distancex = distancexmax
            else:
                distancex = distancexmin
                xfactor = -1
            if(distanceymax <= distanceymin):
                distancey = distanceymax
            else:
                distancey = distanceymin
                yfactor = -1
            if(distancezmax <= distancezmin):
                distancez = distancezmax
                
            else:
                distancez = distancezmin
                zfactor = -1
            xforce, yforce, zforce = math.log(abs(distancex), .05), math.log(abs(distancey), .05), math.log(abs(distancez), .05)
            xforces.append(xfactor*xforce)
            yforces.append(yfactor*yforce)
            zforces.append(zfactor*zforce)
        output = [mean(xforces), mean(yforces), mean(zforces)]
        maximum = 0
        for i in output:
            if(abs(i) > maximum):
                maximum = abs(i)
        output[0] = output[0] / maximum
        output[1] = output[1] / maximum
        output[2] = output[2] / maximum
        return output
    
    def stick(self):
        output = [0, 0, 0]
        midpoint = [0, 0, 0]
        for j in range(3):
            midpoint[j] = sum(self.flock.birds[self.nearestneighbors[i]].position[j] for i in range(7))/len(self.nearestneighbors) 
        output[0], output[1], output[2] = self.position[0] - midpoint[0], self.position[1] - midpoint[1], self.position[2] - midpoint[2]
        maximum = 0
        for i in output:
            if(abs(i) > maximum):
                maximum = abs(i)
        output[0] = output[0] / maximum
        output[1] = output[1] / maximum
        output[2] = output[2] / maximum
        return output
    
    def applyForce(self, initial, force, weight, interval):
        output = initial
        for i in range(3):
            output[i] = output[i] + weight*(force[i]*interval)
        return output
    
    def refreshVelocity(self, variability, sepWeight, stickWeight, boundryWeight, interval):
        """
        Combines multiple forces to output final velocity
        """
        output = self.align(variability)
        output = self.applyForce(output, self.separate(),sepWeight, interval)
        output = self.applyForce(output, self.stick(), stickWeight, interval)
        output = self.applyForce(output, self.wallforce(), boundryWeight, interval)
        self.velocity = output
    
    def refreshPosition(self, interval):
        for i in range(3):
            self.position[i] = self.position[i] + interval * self.velocity[i]
        

In [49]:
class Flock():
    def __init__(self, xmin = 0, xmax = 100, ymin = 0, ymax = 100, zmin = 0, zmax = 100):
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax
        self.zmin = zmin
        self.zmax = zmax
        self.birds = []
        self.size = 0
        
    def center(self):
        center = [0, 0, 0]
        if (self.size == 0):
            center = [(self.xmax - self.xmin)/2, (self.ymax - self.xmin)/ 2, (self.xmax - self.xmin)/2]
        else:  
            for i in range(self.size):
                center[0] += self.birds[i].position[0]
                center[1] += self.birds[i].position[1]
                center[2] += self.birds[i].position[2]
            center[0] = center[0] / self.size
            center[1] = center[1] / self.size
            center[2] = center[2] / self.size
        return center
    
    def distance(self, bird1, bird2):
        distance = math.sqrt(math.pow(bird1.position[0] - bird2.position[0], 2) + math.pow(bird1.position[1] - bird2.position[1], 2) + math.pow(bird1.position[2] - bird2.position[2], 2))
        return distance
        
    def boundries(self):
        return [self.xmin, self.xmax, self.ymin, self.ymax, self.zmin, self.zmax]
        
    def generate(self, size = 100, velocityMin = -10, velocityMax = 10):
        for i in range(size):
            x, y, z = randFloat(self.xmin + self.xmax/20, (self.xmax - self.xmax/20)), randFloat(self.ymin + self.ymax/20, (self.ymax - self.ymax/20)), randFloat(self.zmin + self.zmax/20, (self.zmax - self.zmax/20))
            u, w, v = randFloat(velocityMin, velocityMax), randFloat(velocityMin, velocityMax), randFloat(velocityMin, velocityMax)
            newBird = Bird(id = i, flock = self, position = [x, y, z], velocity = [u, w, v])
            self.birds.append(newBird)
            self.size = len(self.birds)
            del newBird

    def plotConnections(self, limit):
        for mainbird in tqdm(range(limit), unit = 'birds'): 
            for bird in flock.birds[mainbird].nearme():
                x1, x2 = flock.birds[mainbird].position[0], flock.birds[bird].position[0]
                y1, y2 = flock.birds[mainbird].position[1], flock.birds[bird].position[1]
                z1, z2 = flock.birds[mainbird].position[2], flock.birds[bird].position[2]
                x, y, z = np.array([x1, x2]), np.array([y1, y2]), np.array([z1, z2])
                ipv.plot(x, y, z, size = .1)
                time.sleep(.01)
            time.sleep(.1)

    def plot(self):
        flockx, flocky, flockz = np.array([]), np.array([]), np.array([])
        flocku, flockv, flockw = np.array([]), np.array([]), np.array([])
        for bird in range(self.size):
            flockx = np.append(flockx, self.birds[bird].position[0])
            flocky = np.append(flocky, self.birds[bird].position[1])
            flockz = np.append(flockz, self.birds[bird].position[2])
            flocku = np.append(flocku, self.birds[bird].velocity[0])
            flockv = np.append(flockv, self.birds[bird].velocity[1])
            flockw = np.append(flockw, self.birds[bird].velocity[2])
        centerx, centery, centerz = np.array([self.center()[0]]), np.array([self.center()[1]]), np.array([self.center()[1]])
        ipv.clear()
        ipv.xlim(self.xmin, self.xmax)
        ipv.ylim(self.ymin, self.ymax)
        ipv.zlim(self.zmin, self.zmax)
        flock = ipv.quiver(flockx, flocky, flockz, flocku, flockv, flockw, color='black', size = 1.5)
        center = ipv.scatter(centerx, centery, centerz, marker = 'sphere', color = 'blue', size = 2)
        ipv.style.axes_off()
        ipv.style.box_off()
        ipv.show()

    def refreshVelocities(self, variability, sepWeight = 3, stickWeight = 5, boundryWeight = 5, interval = 1):
        for bird in tqdm(self.birds, total = self.size, desc = 'Velocity refreshing...', unit = 'birds'):
            bird.refreshVelocity(variability = variability, sepWeight = sepWeight, stickWeight = stickWeight, boundryWeight = boundryWeight, interval = interval)
            
    def refreshPositions(self, interval = 1):
        for bird in tqdm(self.birds, total = self.size, desc = 'Position refreshing...', unit = 'birds'):
            bird.refreshPosition(interval)
        
    def animationMatrix(self, variability, sepWeight, stickWeight, boundryWeight, length, interval, showCenter):
        x, y, z, u, w, v = np.zeros(shape=(length//interval, self.size)), np.zeros(shape=(length//interval, self.size)), np.zeros(shape=(length//interval, self.size)), np.zeros(shape=(length//interval, self.size)), np.zeros(shape=(length//interval, self.size)), np.zeros(shape=(length//interval, self.size))
        for frame in tqdm(range(length//interval), unit = 'frame', desc = 'Frames completed: '):
            flockx, flocky, flockz, flocku, flockw, flockv = np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([])
            for bird in range(self.size):
                flockx = np.append(flockx, self.birds[bird].position[0])
                flocky = np.append(flocky, self.birds[bird].position[1])
                flockz = np.append(flockz, self.birds[bird].position[2])
                flocku = np.append(flocku, self.birds[bird].velocity[0])
                flockw = np.append(flockw, self.birds[bird].velocity[1])
                flockv = np.append(flockv, self.birds[bird].velocity[2])
            x[frame] = flockx
            y[frame] = flocky
            z[frame] = flockz
            u[frame] = flocku
            w[frame] = flockw
            v[frame] = flockv
            self.refreshVelocities(variability = variability, sepWeight = sepWeight, stickWeight = stickWeight, boundryWeight = boundryWeight, interval = interval)
            self.refreshPositions(interval = interval)
        return x, y, z, u, w, v
        
    def animate(self, variability, sepWeight, stickWeight, boundryWeight, length, interval, showCenter = False):
        x, y, z, u, w, v = self.animationMatrix(variability, sepWeight, stickWeight, boundryWeight, length, interval, showCenter)
        print(x)
        ipv.clear()
        ipv.figure()
        ipv.xlim(self.xmin, self.xmax)
        ipv.ylim(self.ymin, self.ymax)
        ipv.zlim(self.zmin, self.zmax)
        ipv.style.axes_off()
        ipv.style.box_off()
        birds = ipv.quiver(x, y, z, u, w, v, color = 'black', size=1)
        ipv.animation_control(birds) # shows controls for animation controls
        ipv.show()
        
    def __str__(self):
        return ' '.join(str(e) for e in self.birds)


In [51]:
########################
#         TEST         #
########################

flock = Flock()
flock.generate(size = 1500)

flock.animate(1, 6, 3, 4, 50, 1)

# print('The flock center is: ', flock.center())
# flock.plot()
# flock.plotConnections(5)
# flock.refreshVelocities(1.5)
# flock.plot()
# flock.plotConnections(5)

print("Completed")


[[  6.62057375  42.52053796  65.90337233 ...  16.11351407   6.06985919
   84.89570361]
 [  9.42900791  44.92279064  57.93964049 ...  23.2569542   15.15288736
   81.02911244]
 [ 17.38973723  49.48934629  52.77975537 ...  34.31877637  36.86479558
   63.64658659]
 ...
 [ 49.22076332  53.47510829  -4.32111    ...  52.48994128  47.52129915
   32.91061397]
 [ 45.01905031  48.67332194 -13.96881469 ...  41.34166     57.2898848
   32.33045228]
 [ 49.81975515  56.79535596 -16.76073047 ...  47.94491055  56.11289876
   36.32952889]]


Completed
